# 교차 검증과 그리드 서치

## 훈련세트로 모델 훈련하고 테스트 세트로 모델을 평가함 ... 테스트 세트에서 얻은 점수로 어느 정도 성능이 나오겠구나 예측을 한다! 근데... 테스트 세트를 자꾸 사용하다보면 점점 테스트 세트에 맞춰게 되어버림... 

### 가장 좋은 건 테스트 세트를 마지막 딱 한 번 확인하는 게 베스트 

## 검증 세트

### 테스트 세트를 사용하지 않으면 모델이 과소인지 과대인지 확인이 불가능하니까 이를 측정하기 위해서 훈련세트에서 다시 부분을 나눠서 검증용을 만든다 ===> **검증세트**

ex_ 훈련세트 60% 검증세트 20% 테스트세트 20%

In [33]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [34]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [35]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [36]:
# 위에서 나눈 train_input / target을 한 번더 처리해서 20퍼 나눠줌 ... 
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [37]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [38]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

# 훈련용 sub 검증용 val 
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

검증을 한 샘플로 하면 이것도 좀 그러니까 훈련세트에 어느 부분을 랜덤하게 검증으로 쓰는 느낌인가  이거 여러번 반복해서 평균 내는 느낌인듯 

> 훈련세트를 3부분으로 나눠서 교차 검증 수행하는 것을 3-폴드 교차검증 
> k 폴드 교차 검증 이라고 한다 
- 보통 5나 10폴드를 많이 사용함 

In [39]:
from sklearn.model_selection import cross_validate

# cross_validate () 교차 검증 함수 
# 평가할 모델 객체 / 훈련용세트 인풋 / 훈련용 타겟

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00790548, 0.00667334, 0.00674129, 0.00684094, 0.0064435 ]), 'score_time': array([0.00068259, 0.0006237 , 0.00057673, 0.00061011, 0.00061131]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [40]:
import numpy as np
# print(Scores) 보면 test_score가 있음 
# 보통 cross_validate는 5번 수행함 
# test_socre는 5개 평균 점수 
print(np.mean(scores['test_score']))

0.855300214703487


cross_validate()는 섞어서 나누지 않음 
아까 위에서 train_test_split()로 셔플했으니까 상관은 없지만 저 함수는 셔플 안 되니까 저것만 쓸 때는 따로 옵션으로 섞어줘야함 

In [41]:
from sklearn.model_selection import StratifiedKFold

# cv=StratifiedKFold- > kfold 사용해서 섞기 
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [42]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
# 10폴듣 

scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

그리드 서치 
- 적절한 하이퍼 파라미터를 찾아주는 도구 

In [43]:
from sklearn.model_selection import GridSearchCV

# 탐색할 매개변수와 탐색할 값으 리스트를 딕셔너리 형태로 만듦 
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [44]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# min_impurity_Dcrease 값 총 5개 그리고 k-fold 기본 5개니까 5 * 5 = 25 총 25번 실행 
# n_jobs -> 병렬 실행에 필요한 cpu 코어 수지정  기본값은 1 // -1일 때 모든 코어를 사용한다 

In [45]:
gs.fit(train_input, train_target)
# 해당 매개변수를 토대로 5번 실행 

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 파라미터를 찾으면 전체 훈련세트 모델을 다시 만들어야하지만 그리드 서치는 훈련이 끝나면 그 모델 중 검증 점수가 가장 높은 모델의 매개벼수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련함 이 gs 객체는 best_estimator_ 속성에 저앚ㅇ됨 

In [46]:
dt = gs.best_estimator_
#가장 점수가 좋은 모델 
print(dt.score(train_input, train_target))

0.9615162593804117


In [47]:
print(gs.best_params_)
# 가장 좋은 파라미터 

{'min_impurity_decrease': 0.0001}


In [48]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [49]:
# argmax() 해당 리스트 중 가장 ㅡㅋㄴ 값의 인덱스 추출 
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [50]:
params = {
          # 0,0001에서 0.001까지 되도록 0.0001씩 더하기 총 9번 
          'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          # 5에서 20까지 1씩 15번 더하기
          'max_depth': range(5, 20, 1),
          # 2에서 100까지 10씩 10번 더하기
          'min_samples_split': range(2, 100, 10)
          }

          # 9 * 15 * 10 = 6750 개의 모델을 만들고 검사 

In [51]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [52]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [53]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


증가폭을 1혹은 10 0.0.001 등으로 설정을 했는데 더 좁거나 넓은 간격으로 할 수 있지 않을까 = 랜덤서치 

### 랜덤 서치

매개변수 값이 수치일 때 값의 범위나 간격 미리 정하기 힘들다 ... 또 매개변수 조ㄱㄴ이 많으면 그리드 서치 수행 시간이 오래걸림

매개변수 값 목록이 아니라 샘플링 할 수 있는 확률 분표 객체를 전달 ! 


### 싸이파이 
- 적분 ,선형대수, 학률 등 포함한 수치 계산 전용 라이브러리 
- randint - 정수값을 범위에서 고르게 뽑음
- uniform - 실수값을 범위내에서 고르게 뽑음 


In [54]:
from scipy.stats import uniform, randint

In [66]:
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 9, 5, 9, 6, 4, 4, 3, 8, 5])

In [70]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97,  90,  92,  98, 106, 100,  97,  95, 107, 118]))

In [57]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.91045689, 0.29790793, 0.94664968, 0.03428028, 0.70859791,
       0.98552055, 0.968208  , 0.15893275, 0.78871276, 0.01713448])

In [71]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          # 0.0001에서 0.001 까지의 범위 실수값

          'max_depth': randint(20, 50),
          # 20 - 50 범위 정수 
          
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [72]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f41639d0d10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4163deea50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f41971b3210>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f41639d0250>},
                   random_state=42)

In [73]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [74]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [62]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [63]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4163c64350>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4163c64a90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4163e41e10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4163c64a50>},
                   random_state=42)

In [64]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


# 정리 하기 

1. 검증세트
- 하이퍼 파라미터 튜닝을 위해 모델 평가시 테스트 세트 쓰지않기 위해 트레니인셋에서 똑 뗀 것 보통 60 20 20 비율 

2. 교차 검증 
- 훈련세트를 여러 폴드로 나누고 하나를 검증 나머지를 훈련으로 돌림 
- 교차검증은 이 폴드를 다 검증하고 평균을 냄 

3. 그리드 서치 
- 하이퍼 파라미터 탐색을 자동화 해줌 
- 매개변수 리스트를 만들고 가장 좋은 검증 점수 조합을 선택 
- 가장 좋은 조합으로 최종 모델을 훈련시킴! 

4. 랜덤서치 
- 연속된 매개 변수 값 탐색시 사용 
- 직접 나열이 아니라 값을 샘플링 하도록 확률분포객체 전달 
- 랜덤이라 지정 x 

# 패키지 & 함수 

1. cross_validate() 
- 교차 검증응ㄹ 수행 
- cv 매개 변수에 교차 검증 폴드 수나 스플리터 객체 지정 가능 기본값 5
- n_jobs : 사용할 cpu 코어 수 지정 기본 1 / -1 : 가능한 대로 

2. GridSearchCV
- 교차 검증으로 하이퍼 파라미터 탐색 수행 
- 가장 좋은 모델 찾고 훈련세트 전체를 가장 좋은 모델로 훈련시킴 

3. RandomsizedSearchCV
- 교차 검증으로 랜덤한 하이퍼 파라미터 탐색 수행 